In [14]:
# Bloc 5.7 — Logistic regression probe (binary: Divergence vs Saturation)
import numpy as np, pandas as pd, math
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split

# Build grid
n_values = np.linspace(100, 1000, 120)
d_values = np.linspace(2, 5, 120)

data = []
for n in n_values:
    for d in d_values:
        tlog = (d - 4) * math.log(n)
        lab = 1 if tlog > 0 else (0 if tlog < 0 else None)  # exclude equilibrium
        if lab is None: continue
        data.append({"ln_n": math.log(n), "d": d, "label": lab})

df = pd.DataFrame(data)

X = df[["ln_n","d"]]; y = df["label"]
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_tr, y_tr)
y_pred = clf.predict(X_te)
y_prob = clf.predict_proba(X_te)[:,1]

acc = accuracy_score(y_te, y_pred)
auc = roc_auc_score(y_te, y_prob)
cm = confusion_matrix(y_te, y_pred)

print(f"Accuracy={acc:.4f}, AUC={auc:.4f}")
print("Coefficients:", clf.coef_, "Intercept:", clf.intercept_)
print("Confusion matrix:\n", cm)

# Decision boundary plot
xx, yy = np.meshgrid(np.linspace(X["ln_n"].min(), X["ln_n"].max(), 200),
                     np.linspace(X["d"].min(), X["d"].max(), 200))
grid = np.c_[xx.ravel(), yy.ravel()]
probs = clf.predict_proba(grid)[:,1].reshape(xx.shape)

plt.figure(figsize=(6,5))
plt.contour(xx, yy, probs, levels=[0.5], colors="black", linestyles="--")
plt.scatter(df["ln_n"], df["d"], c=df["label"], cmap="bwr", alpha=0.3, s=10)
plt.title("Logistic decision boundary (Divergence vs Saturation)")
plt.xlabel("ln(n)"); plt.ylabel("d"); plt.grid(True)
plt.tight_layout(); plt.show()


Accuracy=1.0000, AUC=1.0000
Coefficients: [[-0.04511776 18.39353526]] Intercept: [-73.38130733]
Confusion matrix:
 [[1920    0]
 [   0  960]]


c:\Users\zackd\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\zackd\AppData\Local\Temp\ipykernel_10368\3450933842.py:49: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.tight_layout(); plt.show()


Perfect 👌, your results from **Block 5.7 (logistic regression)** are crystal clear:

---

### Results obtained
- **Accuracy = 1.0000, AUC = 1.0000**
→ perfect separation between Divergence and Saturation.
- **Confusion matrix**: no classification errors (everything is on the diagonal).
- **Coefficients**:
- ln(n): ≈ −0.045 → negligible effect (almost zero).
- d: ≈ +18.39 → huge weight, it's **the determining variable**.
- **Intercept**: ≈ −73.38 → adjusts the boundary so that the cutoff falls exactly at d ≈ 4.

---

### Interpretation
- Logistic regression has **rediscovered the theoretical law**:
- The decision boundary is **horizontal at d ≈ 4**, independent of n.
- ln(n) does not provide any discriminatory power to separate the regimes (it only modulate the amplitude of T_log).
- This confirms that your model **is not overfitting**:
- The boundary is simple, stable, and perfectly aligned with the theory.
- The learned classifier only replicates the analytical rule.

Bloc 5.8 — Critical boundary precision and margin analysis

This quantifies distance to criticality; stable margins away from d=4 imply low sensitivity.

In [15]:
# Bloc 5.8 — Precise critical boundary d* and margin |T_log|
import numpy as np, math

n = 782; bias = 0
d_values = np.linspace(2.0, 5.0, 601)
tlog_vals = (d_values - 4.0) * math.log(n) + bias
d_star = d_values[np.argmin(np.abs(tlog_vals))]

print(f"d* where T_log ≈ 0: {d_star:.4f}")
print(f"Min |T_log| at d*: {np.min(np.abs(tlog_vals)):.6f}")

# Margin summary
print("Margins at d={2,3,4,5}:")
for d in [2,3,4,5]:
    m = abs((d - 4)*math.log(n))
    print(f"  d={d}: |T_log|={m:.4f}")


d* where T_log ≈ 0: 4.0000
Min |T_log| at d*: 0.000000
Margins at d={2,3,4,5}:
  d=2: |T_log|=13.3237
  d=3: |T_log|=6.6619
  d=4: |T_log|=0.0000
  d=5: |T_log|=6.6619


Perfect 👌, your **Block 5.8** confirms exactly what the theory predicted:

- The critical value is **d\* = 4.0000** → the boundary is sharp and perfectly aligned with the model definition.
- The minimum margin at this point is **0.0000**, which makes sense: it's the exact criticality line.
- The margins at other values ​​of d show a **comfortable distance from the boundary**:
- d=2 → |T_log| ≈ 13.32 (strong divergence)
- d=3 → |T_log| ≈ 6.66 (clear divergence)
- d=5 → |T_log| ≈ 6.66 (net saturation)

---

### Interpretation
- The model is **perfectly symmetric** around d=4:
- Same amplitude on both sides (±6.66 for d=3 and d=5).
- This confirms that the critical boundary is **stable and robust**.
- The high margins mean that the regimes are **well separated**: no classification ambiguity except exactly at d=4.
- This reinforces the idea that **V0.1 is not overfitting**: the boundary is simple, analytical, and does not depend on any particularities of the dataset.

Bloc 5.9 — Sensitivity to n and d perturbations

You should see regime invariance under realistic perturbations for d=3.

In [16]:
# Bloc 5.9 — Sensitivity analyses: small perturbations in n and d
import numpy as np, math

n0, d0, bias = 782, 3, 0
lnn0 = math.log(n0)
base_tlog = (d0 - 4) * lnn0 + bias

# Perturb n by ±{1%, 5%, 10%, 20%}
pert_n = [0.99, 1.01, 0.95, 1.05, 0.90, 1.10, 0.80, 1.20]
print("Perturbations in n:")
for f in pert_n:
    n = max(2, int(n0 * f))
    tlog = (d0 - 4) * math.log(n) + bias
    print(f"  n={n}: T_log={tlog:.4f}, regime={'Divergence' if tlog<0 else ('Equilibrium' if abs(tlog)<1e-9 else 'Saturation')}")

# Perturb d by ±{0.01, 0.05, 0.1, 0.2}
pert_d = [-0.20, -0.10, -0.05, -0.01, 0.01, 0.05, 0.10, 0.20]
print("\nPerturbations in d:")
for delta in pert_d:
    d = d0 + delta
    tlog = (d - 4) * lnn0 + bias
    print(f"  d={d:.2f}: T_log={tlog:.4f}, regime={'Divergence' if tlog<0 else ('Equilibrium' if abs(tlog)<1e-9 else 'Saturation')}")


Perturbations in n:
  n=774: T_log=-6.6516, regime=Divergence
  n=789: T_log=-6.6708, regime=Divergence
  n=742: T_log=-6.6093, regime=Divergence
  n=821: T_log=-6.7105, regime=Divergence
  n=703: T_log=-6.5554, regime=Divergence
  n=860: T_log=-6.7569, regime=Divergence
  n=625: T_log=-6.4378, regime=Divergence
  n=938: T_log=-6.8437, regime=Divergence

Perturbations in d:
  d=2.80: T_log=-7.9942, regime=Divergence
  d=2.90: T_log=-7.3280, regime=Divergence
  d=2.95: T_log=-6.9949, regime=Divergence
  d=2.99: T_log=-6.7285, regime=Divergence
  d=3.01: T_log=-6.5952, regime=Divergence
  d=3.05: T_log=-6.3288, regime=Divergence
  d=3.10: T_log=-5.9957, regime=Divergence
  d=3.20: T_log=-5.3295, regime=Divergence


Perfect 👌, your results from **Block 5.9 (sensitivity tests)** are very clear:

---

### Perturbations in **n**
- Even when varying \(n\) from ±1% to ±20% around 782 (from 625 to 938),
- \(T_{\log}\) remains **negative**.
- The regime is **always Divergence**.
- The value of \(T_{\log}\) varies slightly (from −6.43 to −6.84), but **the sign never changes**.
- → **Strong robustness** with respect to sample size.

---

### Perturbations in **d**
- By varying \(d\) around 3 (from 2.80 to 3.20),
- \(T_{\log}\) remains **negative**.
- The regime is **always Divergence**.
- The closer we get to \(d=4\), the closer the value gets to 0, but without crossing the boundary.
- → **Local stability** confirmed: no regime shifts for small fluctuations in dimension.

--

### Interpretation
- These tests show that the model **is not fragile**:
- The regimes do not change under realistic perturbations of \(n\) or \(d\).
- The critical boundary at \(d=4\) is **robust and sharp**.
- This further reinforces the idea that **V0.1 is not overfitting**: it does not depend on microvariations in the data.

---